# Compute Route-Level Externalities

## Imports, Parameters etc.

In [ ]:
import csv
import json
import os
import traceback

from math import floor

import numpy as np
import plotly
plotly.offline.init_notebook_mode()

In [ ]:
# Parameters
EXPECTED_HEADER = ['ID', 'name', 'polyline_points', 'total_time_in_sec', 'total_distance_in_meters',
                   'number_of_steps', 'maneuvers', 'beauty', 'simplicity', 'pctNonHighwayTime',
                   'pctNonHighwayDist', 'pctNeiTime', 'pctNeiDist']
METRICS = ['time_min', 'dist_km', 'nsteps', 'nsteps_per_km', 'nlefts', 'simplicity', 'beauty',
           'pctNHT', 'pctNHD', 'pctSNT', 'pctSND']
OUTPUT_HEADER = ['filename', 'dist', 'bin', 'num_entries']
for metric in METRICS:
    OUTPUT_HEADER.extend(['LB_{0}'.format(metric), 'avg_{0}'.format(metric), 'UB_{0}'.format(metric)])
FOLDER = "../data/final/impacts"
CITIES = ["lon", "man", "sf", "nyc"]
TYPES = ["rand", "taxi"]

FNS = ["gh_routes_beauty.csv", "gh_routes_fast.csv", "gh_routes_simple.csv", "gh_routes_safety.csv",
       "google_traffic_routes_main_ghmerged.csv", "mapquest_traffic_routes_main_ghmerged.csv"]

id_idx = EXPECTED_HEADER.index("ID")
name_idx = EXPECTED_HEADER.index("name")
time_idx = EXPECTED_HEADER.index("total_time_in_sec")
dist_idx = EXPECTED_HEADER.index("total_distance_in_meters")
step_idx = EXPECTED_HEADER.index("number_of_steps")
man_idx = EXPECTED_HEADER.index("maneuvers")
sim_idx = EXPECTED_HEADER.index("simplicity")
bty_idx = EXPECTED_HEADER.index("beauty")
pctNHT_idx = EXPECTED_HEADER.index("pctNonHighwayTime")
pctNHD_idx = EXPECTED_HEADER.index("pctNonHighwayDist")
pctSND_idx = EXPECTED_HEADER.index("pctNeiDist")
pctSNT_idx = EXPECTED_HEADER.index("pctNeiTime")

## Helper functions for bootstrap resampling

In [ ]:
from random import randint

def resample_vals(values):
    numvals = len(values)
    newvals = []
    for i in range(0, numvals):
        randidx = randint(0, numvals-1)
        newvals.append(values[randidx])
    return newvals
    

def bootstrap_interval(values, numiter=10000, alpha=0.01):
    avg_vals = []
    for i in range(0, numiter):
        newvals = resample_vals(values)
        avg_vals.append(np.mean(newvals))
    sorted_vals = sorted(avg_vals)
    return {'LB' : sorted_vals[int(numiter * (alpha / 2))],
            'avg' : sorted_vals[int(numiter * 0.5)],
            'UB' : sorted_vals[int(numiter * (1 - (alpha / 2)))]}

## Core class for holding and computing statistics

In [ ]:
class RouteStats(object):
    
    def __init__(self, folder, city, routetype, fns, writeoutput):
        self.city = city
        self.input_csvs = ["{0}/{1}_{2}_{3}".format(folder, city, routetype, fn) for fn in fns]
        self.output_csv = "{0}/{1}_{2}_stats.csv".format(folder, city, routetype)
        for i in range(len(self.input_csvs) - 1, -1, -1):
            if not os.path.exists(self.input_csvs[i]):
                 print("{0} does not exist.".format(self.input_csvs.pop(i)))
        self.stats = {}
        self.dist_bins = []
        self.bin_step = 1
        self.highest_bin = 1
        self.straight_line_distances = {}
        self.route_ids = {}
        self.writeoutput = writeoutput
        self.label = '{0}|{1}'.format(city, routetype)
        
    def compute_stats(self):
        # check if valid inputs and set some parameters
        if not self.prepared():
            return

        # build necessary dictionaries
        self.get_straight_line_distances()
        self.get_route_ids()
        self.initialize_stats()
        
        print("Analyzing {0}. Output to {1}".format(self.city, self.output_csv))

        errors = 0
        processed = 0
        for fn in self.input_csvs:
            with open(fn, 'r') as fin:
                csvreader = csv.reader(fin)
                assert next(csvreader)[:len(EXPECTED_HEADER)] == EXPECTED_HEADER
                for line in csvreader:
                    if line[id_idx] not in self.route_ids:
                        continue
                    try:
                        name = line[name_idx]
                        time_min = float(line[time_idx]) / 60
                        dist_km = float(line[dist_idx]) / 1000
                        nlefts = line[man_idx].count('left')
                        nsteps = float(line[step_idx])
                        if 'exit transit' or 'finish' in line[man_idx]:
                            nsteps = nsteps - 1
                        nsteps = nsteps - line[man_idx].count('continue')
                        nsteps_per_km = nsteps / dist_km
                        simplicity = float(line[sim_idx])
                        beauty = float(line[bty_idx])
                        pctNHT = float(line[pctNHT_idx])
                        pctNHD = float(line[pctNHD_idx])
                        pctSNT = float(line[pctSNT_idx])
                        pctSND = float(line[pctSND_idx])
                        dist_bin = self.straight_line_distances[line[id_idx]]
                        self.stats[fn][dist_bin]['avg_time_min'].append(time_min)
                        self.stats[fn][dist_bin]['avg_dist_km'].append(dist_km)
                        self.stats[fn][dist_bin]['avg_nsteps'].append(nsteps)
                        self.stats[fn][dist_bin]['avg_nlefts'].append(nlefts)
                        self.stats[fn][dist_bin]['avg_simplicity'].append(simplicity)
                        self.stats[fn][dist_bin]['avg_beauty'].append(beauty)
                        self.stats[fn][dist_bin]['avg_pctNHT'].append(pctNHT)
                        self.stats[fn][dist_bin]['avg_pctNHD'].append(pctNHD)
                        self.stats[fn][dist_bin]['avg_pctSNT'].append(pctSNT)
                        self.stats[fn][dist_bin]['avg_pctSND'].append(pctSND)
                        processed += 1
                    except (ValueError, ZeroDivisionError, KeyError):
                        errors += 1
                        continue

            self.process_stats(fn)
        print("{0} errors in processing (and thus lines skipped). {1} kept.".format(errors, processed))
        self.update_names()

        # write statistics for each file to an output CSV
        if self.writeoutput:
            with open(self.output_csv, 'w') as fout:
                csvwriter = csv.DictWriter(fout, fieldnames=OUTPUT_HEADER)
                csvwriter.writeheader()
                for dist_bin in self.dist_bins:        
                    for fn in self.input_csvs:
                        if self.stats[fn][dist_bin]['num_entries'] > 0:
                            csvwriter.writerow(self.stats[fn][dist_bin])



                                
    def update_names(self):
        """More readable names for Plotly"""
        for dist_bin in self.dist_bins:
            for fn in self.input_csvs:
                if 'google' in fn:
                    platform = 'Google'
                elif 'mapquest' in fn:
                    platform = 'MapQst'
                elif 'gh' in fn:
                    platform = 'GraphH'
                else:
                    platform = ""
                    print("Don't recognize this filename: {0}".format(filename))

                if 'fast' in fn or 'main' in fn:
                    routetype = "Fastest"
                elif 'beaut' in fn:
                    routetype = "Beautfl"
                elif 'simple' in fn:
                    routetype = "Simplst"
                elif 'saf' in fn:
                    routetype = "Safest"
                else:
                    routetype = ""
                    print("Don't recognize a route type: {0}".format(filename))

                if 'notraffic' in fn:
                    conditions = "(No Traffic)"
                elif 'traffic' in fn:
                    conditions = "(Traffic)"
                else:
                    conditions = ""
                self.stats[fn][dist_bin]['filename'] = "{0}-{1} {2}".format(platform, routetype, conditions)
        
        
    def process_stats(self, fn):
        """Calculate bootstrapped averages and confidence intervals for each stat, distance, and set of routes."""
        print("Bootstrapping CIs for {0}".format(fn))
        for dist_bin in self.dist_bins:
            label = fn
            num_entries = len(self.stats[label][dist_bin]['avg_time_min'])
            self.stats[label][dist_bin]['num_entries'] = num_entries
            if num_entries:
                for metric in METRICS:
                    bootstrapped_interval = bootstrap_interval(self.stats[label][dist_bin]['avg_{0}'.format(metric)],
                                                               numiter=1000, alpha=0.01)
                    self.stats[label][dist_bin]['LB_{0}'.format(metric)] = bootstrapped_interval['LB']
                    self.stats[label][dist_bin]['avg_{0}'.format(metric)] = bootstrapped_interval['avg']
                    self.stats[label][dist_bin]['UB_{0}'.format(metric)] = bootstrapped_interval['UB']
                


    def initialize_stats(self):
        """Initialize data structures for statistics for each distance bin."""
        stats_start_idx = OUTPUT_HEADER.index("num_entries")
        for fn in self.input_csvs:
            self.stats[fn] = {}
            for dist in self.dist_bins:
                self.stats[fn][dist] = {'dist':'{0}-{1} km'.format(dist, dist + self.bin_step), 'filename':fn, 'bin':dist+self.bin_step}
                for stat in OUTPUT_HEADER[stats_start_idx:]:
                    self.stats[fn][dist][stat] = []
            self.stats[fn][self.dist_bins[-1]]['dist'] = '>{0} km'.format(self.dist_bins[-1])



    def get_route_ids(self):
        """Go through all route IDs and only keep the od-pairs that appear in all files."""
        for fn in self.input_csvs:
            with open(fn, 'r') as fin:
                csvreader = csv.reader(fin)
                assert next(csvreader)[:len(EXPECTED_HEADER)] == EXPECTED_HEADER, "{0}: {1}".format(fn, EXPECTED_HEADER)
                for line in csvreader:
                    try:
                        if float(line[dist_idx]) <= 0:
                            continue
                        route_id = line[id_idx]
                        self.route_ids[route_id] = self.route_ids.get(route_id, 0) + 1
                    except IndexError:
                        print("{0}: Skipping {1} {2}".format(fn, len(line), line[0]))
                        continue

        id_list = list(self.route_ids.keys())
        num_files = len(self.input_csvs)
        original_num_ids = len(self.route_ids)
        for route_id in id_list:
            if self.route_ids[route_id] != num_files:
                self.route_ids.pop(route_id)

        print("{0} IDs appear in all files and were kept out of {1}.".format(len(self.route_ids), original_num_ids))

    def prepared(self):
        """Prepare data structures if necessary."""
        if len(self.input_csvs) == 0:
            print("No input CSVs.")
            return False

        self.set_parameters()
        return True

    def set_parameters(self):
        """Set cap to distance bins for analysis."""
        if self.city == "nyc":
            self.highest_bin = 27
        elif self.city == "sf":
            self.highest_bin = 13
        else:
            self.highest_bin = 30
        self.dist_bins = [i for i in range(0, self.highest_bin + 1, self.bin_step)]


    def get_straight_line_distances(self):
        """Assign each od-pair to a distance bin based on Euclidean distance."""
        if 'nyc_' in self.input_csvs[0]:
            if 'taxi' in self.input_csvs[0]:
                fn = "../data/intermediate/nyc_taxi_od_pairs.csv"
            else:
                fn = "../data/intermediate/nyc_rand_od_pairs.csv"
        elif 'sf_' in self.input_csvs[0]:
            if 'taxi' in self.input_csvs[0]:
                fn = "../data/intermediate/sf_taxi_od_pairs.csv"
            else:
                fn = "../data/intermediate/sf_rand_od_pairs.csv"
        elif 'lon_' in self.input_csvs[0]:
            fn = "../data/intermediate/lon_rand_od_pairs.csv"
        elif 'man_' in self.input_csvs[0]:
            fn = "../data/intermediate/man_rand_od_pairs.csv"
        else:
            print("City not detected.")
            return

        with open(fn, 'r') as fin:
            csvreader = csv.reader(fin)
            header = next(csvreader)
            straight_line_idx = header.index("straight_line_distance")
            for line in csvreader:
                dist = float(line[straight_line_idx])
                dist_bin = floor(dist / self.bin_step) * self.bin_step
                if dist_bin >= self.dist_bins[-1]:
                        dist_bin = self.dist_bins[-1]
                self.straight_line_distances[line[id_idx]] = dist_bin

## Down to business - compute stats for each city and set of od pairs

In [ ]:
statsobjs = []
for city in CITIES:
    for routetype in TYPES:
        statsobj = RouteStats(FOLDER, city, routetype, FNS, True)
        try:
            statsobj.compute_stats()
            statsobjs.append(statsobj)
        except Exception:
            traceback.print_exc()
            print("Skipping: {0}, {1}, {2}".format(FOLDER, city, routetype))
            continue

for i in range(len(statsobjs) - 1, -1, -1):
    st = statsobjs[i]
    if len(st.route_ids) == 0:
        statsobjs.pop(i)

## Optional - save output to avoid redoing bootstrapping in future

In [ ]:
import pickle

pickle_fn = "statsobjs_routeexternalities.pkl"

# Write out current stats objects:
pickle.dump(statsobjs, open(pickle_fn, 'wb'))

# Read in previous stats objects:
#with open(pickle_fn, "rb") as fin:
#    statsobjs = pickle.load(fin)

## Compare Random and Taxi routes in NYC and SF

In [ ]:
ylabs = ['avg_nsteps', 'avg_nsteps', 'avg_nsteps_per_km', 'avg_beauty', 'avg_pctNHT', 'avg_pctSNT', 'avg_time_min']
pres_ylabs = ["Avg # Turns", "Avg # Turns", "Avg # Turns<br>per km", "Avg Beauty", "Avg Proportion<br>Time on Highways", "Avg Proportion Time<br>in Small Neighborhoods", "Avg Time (min)"]
pres_city = {'lon':'London', 'man':"Manila", "nyc":"New York City", "sf":"San Francisco"}

for i in range(0, len(pres_ylabs)):
    if i in [1,3]:
        pres_ylabs[i] = pres_ylabs[i] + "<br>(Normalized)"
    pres_ylabs[i] = "<b>{0}</b>".format(pres_ylabs[i])

colors = {'GraphH-Beautfl ' : 'rgb(12,44,132)',
          'GraphH-Simplst ' : 'rgb(34,94,168)',
          'GraphH-Fastest ' : 'rgb(65,182,196)',
          'GraphH-Safest '   : 'rgb(29,145,192)',
          'Google-Fastest (Traffic)' : 'rgb(254,178,76)',
          'MapQst-Fastest (Traffic)' : 'rgb(254,217,118)'}

cities = ['sf', 'nyc'] #, 'lon', 'man']
titles = ['<b>{0}</b>'.format(pres_city[st.city]) for st in statsobjs if 'taxi' not in st.output_csv and st.city in cities]

fig = plotly.tools.make_subplots(rows=len(ylabs), cols=len(titles), subplot_titles=(titles * 1), shared_yaxes=True, shared_xaxes=True, vertical_spacing=0.035)
fig['layout'].update(height=len(ylabs)*640, width=len(titles)*800, showlegend=False, font={'family' : 'Droid Sans', 'size' : 16})
for i in range(0, len(ylabs)):
    traces = {}
    do_normalize = False
    if 'Normalized' in pres_ylabs[i]:
        do_normalize = True
    for statsobj in statsobjs:
        if statsobj.city not in cities:
            continue
        if 'rand' in statsobj.output_csv:
            rt = "rand"
        elif 'taxi' in statsobj.output_csv:
            rt = "taxi"
        traces[statsobj.label] = []
        for fn in statsobj.input_csvs:
            if ylabs[i] == 'avg_time_min' and ('google' in fn or 'mapquest' in fn):
                continue
            xvals = []
            yvals = []
            yvals_lb = []
            yvals_ub = []
            desc = ""
            for dist_bin in statsobj.dist_bins:
                normalize = 1
                if do_normalize and ylabs[i] == 'avg_dist_km':
                    #normalize = 1
                    #normalize = statsobj.stats['{0}/{1}_{2}_{3}.csv'.format(FOLDER, statsobj.city, TYPES[0], 'gh_routes_shortest')][dist_bin][ylabs[i]]
                    normalize = statsobj.stats['{0}/{1}_{2}_{3}.csv'.format(FOLDER, statsobj.city, rt, 'gh_routes_fast')][dist_bin][ylabs[i]]
                elif do_normalize and ylabs[i] == 'avg_time_min':
                    #normalize = 1
                    normalize = statsobj.stats['{0}/{1}_{2}_{3}.csv'.format(FOLDER, statsobj.city, rt, 'gh_routes_fast')][dist_bin][ylabs[i]]
                elif do_normalize and (ylabs[i] == 'avg_simplicity' or ylabs[i] == 'avg_nlefts' or ylabs[i] == 'avg_nsteps'):
                    normalize = statsobj.stats['{0}/{1}_{2}_{3}.csv'.format(FOLDER, statsobj.city, rt, 'gh_routes_simple')][dist_bin][ylabs[i]]
                elif do_normalize and ylabs[i] == 'avg_beauty':
                    normalize = statsobj.stats['{0}/{1}_{2}_{3}.csv'.format(FOLDER, statsobj.city, rt, 'gh_routes_beauty')][dist_bin][ylabs[i]]
                if statsobj.stats[fn][dist_bin]['num_entries'] > 0:
                    xvals.append(dist_bin)
                    avg = statsobj.stats[fn][dist_bin][ylabs[i]] / normalize
                    lb = avg - (statsobj.stats[fn][dist_bin][ylabs[i].replace("avg_", "LB_")] / normalize)
                    ub = (statsobj.stats[fn][dist_bin][ylabs[i].replace("avg_", "UB_")] / normalize) - avg
                    if ylabs[i] == 'avg_pctNHT' or ylabs[i] == 'avg_pctNHD':
                        avg = 1 - avg
                        lb = 1 - ub
                        ub = 1 - lb
                    yvals.append(avg)
                    # TODO: figure out how to compute this correctly at the bootstrap step and not at this already aggregated step
                    yvals_lb.append(lb)
                    yvals_ub.append(ub)
                    desc = "{0}|{1}".format(statsobj.label, statsobj.stats[fn][dist_bin]['filename'])  # lots of overwriting, but simplest method
                    
            if xvals:
                trace = plotly.graph_objs.Scatter(x=xvals,
                                                  y=yvals,
                                                  name=desc,
                                                  mode='lines',
                                                  line={'color':colors[desc.split("|")[2]], 'width':2},
                                                  error_y={'type' : "data",
                                                           'array' : yvals_ub,
                                                           'arrayminus' : yvals_lb,
                                                           'color':colors[desc.split("|")[2]],
                                                           'width':0.5,
                                                           'visible' : True,
                                                           'symmetric' : True})
                traces[statsobj.label].append(trace)
                
    in_line_with_plotly = 1
    for j in range(0, len(statsobjs)):
        if traces.get(statsobjs[j].label, False):# and 'rand' not in statsobjs[j].output_csv:
            for trace in traces[statsobjs[j].label]:
                if statsobjs[j].city == "sf":
                    fig.append_trace(trace, i+1, 1)
                else:
                    fig.append_trace(trace, i+1, 2)
                #fig.append_trace(trace, i+1, j+in_line_with_plotly)
        else:
            in_line_with_plotly = in_line_with_plotly - 1
    fig['layout']['yaxis{0}'.format(i+1)].update(title=pres_ylabs[i])
    #fig['layout']['yaxis{0}'.format(i+1)]['titlefont'].update(size=24, family="Droid Sans")
    fig['layout']['yaxis{0}'.format(i+1)]['domain'][0] = max(0, fig['layout']['yaxis{0}'.format(i+1)]['domain'][0])

for j in range(0, len(titles)):
    fig['layout']['xaxis{0}'.format(j+1)].update(title="")
    #fig['layout']['xaxis{0}'.format(j+1)]['titlefont'].update(size=24, family="Droid Sans")
fig['layout']['xaxis{0}'.format(3)].update(title="<b>Straight-Line Distance (km)</b>")
for ann in fig['layout']['annotations']:
    ann['font'] = {'size':16, 'family':'Droid Sans'}
plotly.offline.iplot(fig, show_link=False, image="png", image_height=1400, image_width=1600, filename="impacts")

## Compare route-level externalities for random od-pairs across all cities

In [ ]:
ylabs = ['avg_nsteps', 'avg_nsteps', 'avg_nsteps_per_km', 'avg_beauty', 'avg_pctNHT', 'avg_pctSNT', 'avg_time_min']
pres_ylabs = ["Avg # Turns", "Avg # Turns", "Avg # Turns<br>per km", "Avg Beauty", "Avg Proportion<br>Time on Highways", "Avg Proportion Time<br>in Small Neighborhoods", "Avg Time (min)"]
pres_city = {'lon':'London', 'man':"Manila", "sin":"Singapore", "nyc":"New York City", "sf":"San Francisco"}

for i in range(0, len(pres_ylabs)):
    if i in [1,3]:
        pres_ylabs[i] = pres_ylabs[i] + "<br>(Normalized)"
    pres_ylabs[i] = "<b>{0}</b>".format(pres_ylabs[i])

colors = {'GraphH-Beautfl ' : 'rgb(12,44,132)',
          'GraphH-Simplst ' : 'rgb(34,94,168)',
          'GraphH-Fastest ' : 'rgb(65,182,196)',
          'GraphH-Safest '   : 'rgb(29,145,192)',
          'Google-Fastest (Traffic)' : 'rgb(254,178,76)',
          'MapQst-Fastest (Traffic)' : 'rgb(254,217,118)'}

cities = ['sf', 'nyc', 'lon', 'man']
titles = ['<b>{0}</b>'.format(pres_city[st.city]) for st in statsobjs if 'taxi' not in st.output_csv and st.city in cities]

fig = plotly.tools.make_subplots(rows=len(ylabs), cols=len(titles), subplot_titles=(titles * 1), shared_yaxes=True, shared_xaxes=True, vertical_spacing=0.035)
fig['layout'].update(height=len(ylabs)*480, width=len(titles)*600, showlegend=False, font={'family' : 'Droid Sans', 'size' : 16})
for i in range(0, len(ylabs)):
    traces = {}
    do_normalize = False
    if 'Normalized' in pres_ylabs[i]:
        do_normalize = True
    for statsobj in statsobjs:
        if 'rand' in statsobj.output_csv or statsobj.city not in cities:
            continue
        traces[statsobj.label] = []
        for fn in statsobj.input_csvs:
            if ylabs[i] == 'avg_time_min' and ('google' in fn or 'mapquest' in fn):
                continue
            xvals = []
            yvals = []
            yvals_lb = []
            yvals_ub = []
            desc = ""
            for dist_bin in statsobj.dist_bins:
                normalize = 1
                if do_normalize and ylabs[i] == 'avg_dist_km':
                    #normalize = 1
                    #normalize = statsobj.stats['{0}/{1}_{2}_{3}.csv'.format(FOLDER, statsobj.city, TYPES[0], 'gh_routes_shortest')][dist_bin][ylabs[i]]
                    normalize = statsobj.stats['{0}/{1}_{2}_{3}.csv'.format(FOLDER, statsobj.city, "grid", 'gh_routes_fast')][dist_bin][ylabs[i]]
                elif do_normalize and ylabs[i] == 'avg_time_min':
                    #normalize = 1
                    normalize = statsobj.stats['{0}/{1}_{2}_{3}.csv'.format(FOLDER, statsobj.city, "grid", 'gh_routes_fast')][dist_bin][ylabs[i]]
                elif do_normalize and (ylabs[i] == 'avg_simplicity' or ylabs[i] == 'avg_nlefts' or ylabs[i] == 'avg_nsteps'):
                    normalize = statsobj.stats['{0}/{1}_{2}_{3}.csv'.format(FOLDER, statsobj.city, "grid", 'gh_routes_simple')][dist_bin][ylabs[i]]
                elif do_normalize and ylabs[i] == 'avg_beauty':
                    normalize = statsobj.stats['{0}/{1}_{2}_{3}.csv'.format(FOLDER, statsobj.city, "grid", 'gh_routes_beauty')][dist_bin][ylabs[i]]
                if statsobj.stats[fn][dist_bin]['num_entries'] > 0:
                    xvals.append(dist_bin)
                    avg = statsobj.stats[fn][dist_bin][ylabs[i]] / normalize
                    lb = avg - (statsobj.stats[fn][dist_bin][ylabs[i].replace("avg_", "LB_")] / normalize)
                    ub = (statsobj.stats[fn][dist_bin][ylabs[i].replace("avg_", "UB_")] / normalize) - avg
                    if ylabs[i] == 'avg_pctNHT' or ylabs[i] == 'avg_pctNHD':
                        avg = 1 - avg
                        lb = 1 - ub
                        ub = 1 - lb
                    yvals.append(avg)
                    # TODO: figure out how to compute this correctly at the bootstrap step and not at this already aggregated step
                    yvals_lb.append(lb)
                    yvals_ub.append(ub)
                    desc = statsobj.stats[fn][dist_bin]['filename']  # lots of overwriting, but simplest method
                    
            if xvals:
                trace = plotly.graph_objs.Scatter(x=xvals,
                                                  y=yvals,
                                                  name=desc,
                                                  mode='lines',
                                                  line={'color':colors[desc]},
                                                  error_y={'type' : "data",
                                                           'array' : yvals_ub,
                                                           'arrayminus' : yvals_lb,
                                                           'color':colors[desc],
                                                           'width':0.5,
                                                           'visible' : True,
                                                           'symmetric' : True})
                traces[statsobj.label].append(trace)
                
    in_line_with_plotly = 1
    for j in range(0, len(statsobjs)):
        if traces.get(statsobjs[j].label, False) and 'rand' not in statsobjs[j].output_csv:
            for trace in traces[statsobjs[j].label]:
                fig.append_trace(trace, i+1, j+in_line_with_plotly)
        else:
            in_line_with_plotly = in_line_with_plotly - 1
    fig['layout']['yaxis{0}'.format(i+1)].update(title=pres_ylabs[i])
    #fig['layout']['yaxis{0}'.format(i+1)]['titlefont'].update(size=24, family="Droid Sans")
    fig['layout']['yaxis{0}'.format(i+1)]['domain'][0] = max(0, fig['layout']['yaxis{0}'.format(i+1)]['domain'][0])

for j in range(0, len(titles)):
    fig['layout']['xaxis{0}'.format(j+1)].update(title="")
    #fig['layout']['xaxis{0}'.format(j+1)]['titlefont'].update(size=24, family="Droid Sans")
fig['layout']['xaxis{0}'.format(3)].update(title="<b>Straight-Line Distance (km)</b>")
for ann in fig['layout']['annotations']:
    ann['font'] = {'size':16, 'family':'Droid Sans'}
plotly.offline.iplot(fig, show_link=False, image="png", image_height=1600, image_width=1600, filename="impacts")